In [ ]:
from rwa_model import RWA
from utils import AddTask

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import DataLoader

import numpy as np

import matplotlib.pyplot as plt
from tensorboard_logger import configure, log_value

configure("training/lr_decay_low_diff_dropout")

In [ ]:
num_features = 2
num_classes = 1
num_cells = 250
batch = 100
rwa = RWA(num_features, num_cells, num_classes, fwd_type="cumulative")  # something is bad

criterion = nn.MSELoss()

print_steps = 5

current_lr = 0.01

running_loss = 0.0
time_since_decay = 0
accumulated_loss = []

test = AddTask(100000, 10000, 100)

data_loader = DataLoader(test, batch_size=batch, shuffle=True, num_workers=4)

s, n, d, h, a_max = rwa.init_sndha(batch)
rwa.register_parameter('s', s)  # make sure s changes after each optimizer step

optimizer = optim.Adam(rwa.parameters(), lr=current_lr)

rwa.train()

for epoch in range(10):
    
    for i, data in enumerate(data_loader, 0):

        inputs, labels = data
        inputs = Variable(inputs)
        labels = Variable(labels)

        rwa.zero_grad()
        outputs, rwa.s, n_new, d_new, h_new, a_newmax = rwa(inputs, rwa.s, n, d, h, a_max)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        n = Variable(n_new.data)
        d = Variable(d_new.data)
        h = Variable(h_new.data)
        a_max = Variable(a_newmax.data)

        running_loss += loss.data[0]
        accumulated_loss.append(loss.data[0])
        time_since_decay += 1

        if i % print_steps == print_steps-1:
            current_step = i + 1 + len(data_loader) * epoch
            print("Current step: ", current_step, "Loss: ", running_loss / print_steps)
            log_value("Loss", running_loss / print_steps, step=current_step)
            log_value("LR", current_lr, step=current_step)
            log_value("Error", np.abs(outputs.data[0, 0] - labels.data[0, 0]), step=current_step)
            log_value("Output", outputs.data[0, 0], step=current_step)
            
            if np.abs(np.mean(np.diff(accumulated_loss))) <= current_lr:
                torch.save(rwa.state_dict(), "models/add.dat")
                current_lr = max([current_lr * 1e-1, 1e-8])
                print("lr decayed to: ", current_lr)
                optimizer = optim.Adam(rwa.parameters(), lr=current_lr)
                accumulated_loss.clear()
                time_since_decay = 0
                
#                 if np.mean(accumulated_loss) >= 0.165:
#                     torch.save(rwa.state_dict(), "models/add.dat")
#                     current_lr = min([current_lr * 2, 1e-2])
#                     print("lr decayed to: ", current_lr)
#                     optimizer = optim.Adam(rwa.parameters(), lr=current_lr)
#                     accumulated_loss.clear()
#                     time_since_decay = 0
                    
#                 else:
#                     torch.save(rwa.state_dict(), "models/add.dat")
#                     current_lr = max([current_lr * 1e-1, 1e-8])
#                     print("lr decayed to: ", current_lr)
#                     optimizer = optim.Adam(rwa.parameters(), lr=current_lr)
#                     accumulated_loss.clear()
#                     time_since_decay = 0
                
                
            running_loss = 0.0

torch.save(rwa.state_dict(), "models/rwa_add.dat")
print("Finished Training")

Current step:  5 Loss:  0.6890386506915093
Current step:  10 Loss:  0.6728095769882202
Current step:  15 Loss:  0.5481717526912689
Current step:  20 Loss:  0.40343273878097535
Current step:  25 Loss:  0.2074165940284729
Current step:  30 Loss:  0.2280484437942505
Current step:  35 Loss:  0.17410408854484558
Current step:  40 Loss:  0.23224450647830963
Current step:  45 Loss:  0.1907748430967331
Current step:  50 Loss:  0.17685036063194276
Current step:  55 Loss:  0.17783498466014863
Current step:  60 Loss:  0.16261211037635803
Current step:  65 Loss:  0.17822770178318023
Current step:  70 Loss:  0.1659500002861023
Current step:  75 Loss:  0.18270912766456604
Current step:  80 Loss:  0.16471443176269532
Current step:  85 Loss:  0.16926828622817994
Current step:  90 Loss:  0.15116122215986252
Current step:  95 Loss:  0.18050384819507598
Current step:  100 Loss:  0.16543774902820588
Current step:  105 Loss:  0.17910849153995514
lr decayed to:  0.001
Current step:  110 Loss:  0.16986500322